In [269]:
from sympy import symbols, Symbol, Function, Transpose, sqrt
from sympy import sin, cos
from sympy.matrices import Matrix, eye, zeros, ones, diag
import sympy
import numpy as np
import matplotlib.pyplot as plt

We start with a potential

$$
U(r) = (r - r_0)(a + b r + c r^2 + d r^3)
$$

and impose the following conditions

$$
\begin{aligned}
U(0) & = U_0 & \text{(Barrier height at center)}\\
U'(0) & = 0 & \text{(Center is extremum)} \\
U'(r_0) & = 0 & \text{(Transition path is extremum)} \\
U''(r_0) & = k & \text{(Transition path has width k)}
\end{aligned}
$$

we get

$$
\begin{aligned}
a = - \frac{U_0}{r_0},\ & b = - \frac{U_0}{r_0^2} \\
c = \frac{10 U_0 - k r_0^2}{2 r_0^3},\ & d = \frac{k r_0^2 - 6 U_0}{2 r_0^4}
\end{aligned}
$$


and thus


$$
U(r) = \frac{1}{2 r_0^4} (r - r_0)^2 [ k r^2 r_0^2 - 2 U_0 (r - r_0)(3r + r_0) ]
$$

In order to ensure that the potential has the appropriate sombrero shape, we must have

$$
\begin{aligned}
U''(0) & < 0 & \text{(Center is maximum)}\\
\lim_{r \to \infty} U(r) & = \infty & \text{(Potential is confining)}
\end{aligned}
$$

This imposes the following condition on the curvature of the transition path

$$
\frac{6 U_0}{r_0^2} \leq k \leq \frac{12 U_0}{r_0^2} \quad (1)
$$

For $k > \frac{12 U_0}{r_0^2}$ the central extremum becomes a minimum. For $k < \frac{6 U_0}{r_0^2}$, a second maximum emerges after $r_0$, meaning that $U(r) \to -\infty$ as $r \to \infty$.

In other words we must have

$$
k(\theta) = \frac{6 U_0}{r_0^2} (1 + 2 h( \theta ) )
$$

where $0 \leq h(\theta) \leq 1$.

We set

$$
h(\theta) = \frac{1}{4}( \xi_2 + \xi_1 + (\xi_2 - \xi_1) \sin(\theta) )
$$

such that $h(-\pi/2) = \xi_1$ and $h(\pi/2) = \xi_2$, and thus $k(-\pi/2) = \frac{6 U_0}{r_0^2} (1 + \xi_1)$ and $k(\pi/2) = \frac{6 U_0}{r_0^2} (1 + \xi_2)$.

Finally, we have $\theta = \text{atan2}(x, y)$, and so $\sin(\theta) = \frac{y}{r}$. The final expression for $k$ is

$$
k = \frac{3 U_0}{r_0^2} \left( 2 + \xi_+ + \xi_- \frac{y}{r} \right)
$$

where $\xi_\pm = \xi_2 \pm \xi_1$.

### Construct potential

In [359]:
x = Symbol('x')
y = Symbol('y')
r = Symbol('r')
theta = Symbol('theta')
xip = Symbol('xi_p')
xim = Symbol('xi_m')
k = Function('k')(r,theta)
#k = (3*U0 / r0**2) * (2 + xip + xim*y/r)

U = (1 / (2*r0**4))*(r - r0)**2 * (k*r**2*r0**2 - 2*U0*(r-r0)*(3*r + r0))

F = -Matrix([U.diff(r), U.diff(theta)/r])
F[0] = F[0].simplify()
F[1] = F[1].simplify()
divF = (r*F[0]).diff(r)/r + F[1].diff(theta)/r
divF = divF.simplify().simplify()

In [360]:
divF.simplify().expand()

48*U0*r**2/r0**4 - 72*U0*r/r0**3 + 24*U0/r0**2 - r**4*Derivative(k(r, theta), (r, 2))/(2*r0**2) + r**3*Derivative(k(r, theta), (r, 2))/r0 - 9*r**3*Derivative(k(r, theta), r)/(2*r0**2) - r**2*Derivative(k(r, theta), (r, 2))/2 + 7*r**2*Derivative(k(r, theta), r)/r0 - 8*r**2*k(r, theta)/r0**2 - r**2*Derivative(k(r, theta), (theta, 2))/(2*r0**2) - 5*r*Derivative(k(r, theta), r)/2 + 9*r*k(r, theta)/r0 + r*Derivative(k(r, theta), (theta, 2))/r0 - 2*k(r, theta) - Derivative(k(r, theta), (theta, 2))/2

### FW instanton

In [95]:
T = Symbol('T')
t = Symbol('t')
beta = Symbol('beta')
gamma = Symbol('gamma')

k_expr = (3*U0 / r0**2) * (2 + xip + xim*sympy.sin(theta))

inst_r = r0
inst_theta = sympy.pi * (1 - t/ T)

xdot = inst_r.diff(t) * sympy.cos(inst_theta) - inst_r * sympy.sin(inst_theta) * inst_theta.diff(t)
ydot = inst_r.diff(t) * sympy.sin(inst_theta) + inst_r * sympy.cos(inst_theta) * inst_theta.diff(t)

Lfw = (beta*gamma/4) * ( (xdot - F[0])**2 + (ydot - F[1])**2 ).simplify()
Lfw = Lfw.subs(k, k_expr)
Lfw = Lfw.subs(r, inst_r).subs(theta, inst_theta)
Lfw = Lfw.simplify()

Sfw = sympy.integrate(Lfw, (t, 0, T)).simplify()
Sfw

pi**2*beta*gamma*r0**2/(2*T)

OM action of upper path

In [96]:
inst_r = r0
inst_theta = sympy.pi * (1 - t/ T)

Lfw = (beta*gamma/4) * ( (xdot - F[0])**2 + (ydot - F[1])**2 ).simplify()
Lfw = Lfw.subs(k, k_expr)
Lfw = Lfw.subs(r, inst_r).subs(theta, inst_theta)
Lfw = Lfw.simplify()

Lom = Lfw
Lom += (1/(2*gamma)) * divF
Lom = Lom.subs(k, k_expr)
Lom = Lom.subs(r, inst_r).subs(theta, inst_theta)
Lom = Lom.simplify()

Som_upper = sympy.integrate(Lom, (t, 0, T)).simplify()
Som_upper

-3*T*U0*xi_m/(pi*gamma) - 3*T*U0*xi_p/(2*gamma) - 3*T*U0/gamma + pi**2*beta*gamma*r0**2/(2*T)

OM action of lower path

In [99]:
inst_r = r0
inst_theta = sympy.pi * (1 + t/ T)

Lfw = (beta*gamma/4) * ( (xdot - F[0])**2 + (ydot - F[1])**2 ).simplify()
Lfw = Lfw.subs(k, k_expr)
Lfw = Lfw.subs(r, inst_r).subs(theta, inst_theta)
Lfw = Lfw.simplify()

Lom = Lfw
Lom += (1/(2*gamma)) * divF
Lom = Lom.subs(k, k_expr)
Lom = Lom.subs(r, inst_r).subs(theta, inst_theta)
Lom = Lom.simplify()

Som_lower = sympy.integrate(Lom, (t, 0, T)).simplify()
Som_lower

3*T*U0*xi_m/(pi*gamma) - 3*T*U0*xi_p/(2*gamma) - 3*T*U0/gamma + pi**2*beta*gamma*r0**2/(2*T)

In [100]:
delta_Som = (Som_upper - Som_lower).simplify()

delta_Som

-6*T*U0*xi_m/(pi*gamma)

## 2nd variation

In [185]:
T = Symbol('T')
t = Symbol('t')
beta = Symbol('beta')
gamma = Symbol('gamma')

x = Function('x')(t)
y = Function('y')(t)
xdot = Function('xdot')(t)
ydot = Function('ydot')(t)

r = Function('r')(x,y)
xip = Symbol('xi_p')
xim = Symbol('xi_m')
#k = Function('k')(x,y)
k = (3*U0 / r0**2) * (2 + xip + xim*y/r)

U = (1 / (2*r0**4))*(r - r0)**2 * (k*r**2*r0**2 - 2*U0*(r-r0)*(3*r + r0))

F = -Matrix([U.diff(x), U.diff(y)])
F = F.subs(r.diff(x), x/r).subs(r.diff(y), y/r)
F[0] = F[0].simplify()
F[1] = F[1].simplify()
divF = (F[0].diff(x) + F[1].diff(y))
divF = divF.subs(r.diff(x), x/r).subs(r.diff(y), y/r)
divF = divF.simplify().simplify()

In [187]:
L_FW = (beta*gamma/4) * ( (xdot - F[0])**2 + (ydot - F[1])**2 ).simplify()
L_OM = L_FW
L_OM += (1/(2*gamma)) * divF
L_OM = L_OM.simplify()

In [191]:
coords = [x, y]
dcoords = [xdot, ydot]

L = L_FW

L_x_x = zeros(2,2)
for i in range(2):
    for j in range(2):
        L_x_x[i,j] = L.diff(coords[i]).subs(r.diff(x), x/r).subs(r.diff(y), y/r).simplify()
        L_x_x[i,j] = L_x_x[i,j].diff(coords[j]).subs(r.diff(x), x/r).subs(r.diff(y), y/r).simplify()
        
L_x_dx = zeros(2,2)
for i in range(2):
    for j in range(2):
        L_x_dx[i,j] = L.diff(coords[i]).subs(r.diff(x), x/r).subs(r.diff(y), y/r).simplify()
        L_x_dx[i,j] = L_x_dx[i,j].diff(dcoords[j]).simplify()
        
        
L_dx_dx = zeros(2,2)
for i in range(2):
    for j in range(2):
        L_dx_dx[i,j] = L.diff(dcoords[i]).simplify().diff(dcoords[j]).simplify()

In [367]:
L_x_dx[0,0].expand()

3*U0*beta*gamma*xi_m*y(t)/(4*r(x(t), y(t))) - 3*U0*beta*gamma*xi_m*x(t)**2*y(t)/(4*r(x(t), y(t))**3) + 3*U0*beta*gamma*xi_p/2 - 3*U0*beta*gamma - 3*U0*beta*gamma*xi_m*y(t)/r0 - 9*U0*beta*gamma*xi_p*r(x(t), y(t))/(2*r0) - 9*U0*beta*gamma*xi_p*x(t)**2/(2*r0*r(x(t), y(t))) + 3*U0*beta*gamma*r(x(t), y(t))/r0 + 3*U0*beta*gamma*x(t)**2/(r0*r(x(t), y(t))) + 9*U0*beta*gamma*xi_m*r(x(t), y(t))*y(t)/(4*r0**2) + 9*U0*beta*gamma*xi_m*x(t)**2*y(t)/(4*r0**2*r(x(t), y(t))) + 3*U0*beta*gamma*xi_p*r(x(t), y(t))**2/r0**2 + 6*U0*beta*gamma*xi_p*x(t)**2/r0**2

Compute 2nd variation around FW instanton

In [290]:
ell = Symbol('ell', integer=True, nonzero=True)
m = Symbol('m', integer=True, nonzero=True)

lambda_ell = T**2 / (sympy.pi**2 * ell**2)
phi_ell = sqrt(2/T) * sympy.sin(t / (T / (sympy.pi * ell)))
phi_m = sqrt(2/T) * sympy.sin(t / (T / (sympy.pi * m)))

In [305]:
sympy.integrate(ui_L_x_x[0,0].expand().args[0]*phi_m*phi_ell, (t, 0, T), manual=True).doit()

KeyboardInterrupt: 

In [ ]:
sympy.integrate(ui_L_x_x[0,0]*phi_m*phi_ell, (t, 0, T))

In [292]:
ui_L_x_x[0,0].expand().args[0]*phi_m*phi_ell

36*U0**2*beta*gamma*sin(pi*ell*t/T)*sin(pi*m*t/T)*cos(pi*t/T)**2/T

In [281]:
from sympy.integrals.trigonometry import trigintegrate

In [352]:


sympy.integrate(sin(a*t)*sin(2*t)*cos(t)**2, t)

KeyboardInterrupt: 

In [338]:
sympy.integrate(sin(a*c)*sin(2*c)*cos(c)**2, c)

KeyboardInterrupt: 

In [350]:
sympy.integrate(sin(a*t)*sin(b*t)*cos(c*t)**2, t)

KeyboardInterrupt: 

In [344]:
ui_L_x_x[0,0].expand().args[0]*phi_m*phi_ell

36*U0**2*beta*gamma*sin(pi*ell*t/T)*sin(pi*m*t/T)*cos(pi*t/T)**2/T

In [348]:
ui_L_x_x[0,1].expand()

-9*U0**2*beta*gamma*xi_m**2*sin(pi*t/T)**2*sin(2*pi*t/T)/4 - 9*U0**2*beta*gamma*xi_m*xi_p*sin(pi*t/T)*sin(2*pi*t/T)/2 - 9*U0**2*beta*gamma*xi_m*sin(pi*t/T)*sin(2*pi*t/T) - 9*U0**2*beta*gamma*xi_p**2*sin(2*pi*t/T)/4 - 9*U0**2*beta*gamma*xi_p*sin(2*pi*t/T) - 9*U0**2*beta*gamma*sin(2*pi*t/T) + 3*pi*U0*beta*gamma*xi_m*sin(pi*t/T)**3/(2*T) - 3*pi*U0*beta*gamma*xi_m*sin(pi*t/T)*cos(pi*t/T)**2/T + 3*pi*U0*beta*gamma*xi_p*sin(pi*t/T)**2/(2*T) - 3*pi*U0*beta*gamma*xi_p*cos(pi*t/T)**2/(2*T) + 3*pi*U0*beta*gamma*sin(pi*t/T)**2/T - 3*pi*U0*beta*gamma*cos(pi*t/T)**2/T

In [297]:
a, b, c = sympy.symbols('a b c', nonzero=True)

res = trigintegrate(sin(a * t) * sin(b * t) * cos(c * t)**2, t)

res

In [349]:
trigintegrate()

<function sympy.integrals.trigonometry.trigintegrate(f, x, conds='piecewise')>

In [347]:
ui_L_x_dx[0,0].expand()

3*U0*beta*gamma*xi_m*sin(pi*t/T)*cos(pi*t/T)**2/2 + 3*U0*beta*gamma*xi_p*cos(pi*t/T)**2/2 + 3*U0*beta*gamma*cos(pi*t/T)**2

In [372]:
ui_L_x_x[0,0].simplify()

-3*U0*beta*gamma*(-3*T*U0*(xi_m*sin(pi*t/T) + xi_p + 2)**2*cos(pi*t/T) + 2*pi*xi_m*sin(pi*t/T)**2 - pi*xi_m*cos(pi*t/T)**2 + 2*pi*xi_p*sin(pi*t/T) + 4*pi*sin(pi*t/T))*cos(pi*t/T)/(2*T)

In [254]:
upper_inst_r = r0
upper_inst_theta = sympy.pi * (1 - t/ T)
upper_inst_xdot = upper_inst_r.diff(t) * sympy.cos(upper_inst_theta) - upper_inst_r * upper_inst_theta.diff(t) * sympy.sin(upper_inst_theta)
upper_inst_ydot = upper_inst_r.diff(t) * sympy.sin(upper_inst_theta) + upper_inst_r * upper_inst_theta.diff(t) * sympy.cos(upper_inst_theta)

ui_L_x_x = L_x_x
ui_L_x_x = ui_L_x_x.subs(r, upper_inst_r)
ui_L_x_x = ui_L_x_x.subs(x, r0*sympy.cos(upper_inst_theta))
ui_L_x_x = ui_L_x_x.subs(y, r0*sympy.sin(upper_inst_theta))
ui_L_x_x = ui_L_x_x.subs(xdot, upper_inst_xdot)
ui_L_x_x = ui_L_x_x.subs(ydot, upper_inst_ydot)

ui_L_x_dx = L_x_dx
ui_L_x_dx = ui_L_x_dx.subs(r, upper_inst_r)
ui_L_x_dx = ui_L_x_dx.subs(x, r0*sympy.cos(upper_inst_theta))
ui_L_x_dx = ui_L_x_dx.subs(y, r0*sympy.sin(upper_inst_theta))
ui_L_x_dx = ui_L_x_dx.subs(xdot, upper_inst_xdot)
ui_L_x_dx = ui_L_x_dx.subs(ydot, upper_inst_ydot)

ui_L_dx_dx = L_dx_dx
ui_L_dx_dx = ui_L_dx_dx.subs(r, upper_inst_r)
ui_L_dx_dx = ui_L_dx_dx.subs(x, r0*sympy.cos(upper_inst_theta))
ui_L_dx_dx = ui_L_dx_dx.subs(y, r0*sympy.sin(upper_inst_theta))
ui_L_dx_dx = ui_L_dx_dx.subs(xdot, upper_inst_xdot)
ui_L_dx_dx = ui_L_dx_dx.subs(ydot, upper_inst_ydot)

for i in range(2):
    for j in range(2):
        ui_L_x_x[i,j] = ui_L_x_x[i,j].simplify()
        ui_L_x_dx[i,j] = ui_L_x_dx[i,j].simplify()
        ui_L_dx_dx[i,j] = ui_L_dx_dx[i,j].simplify()
        
ui_L_dx_dx

Matrix([
[beta*gamma/2,            0],
[           0, beta*gamma/2]])